In [50]:
with open("day_4.txt", "r") as f:
    inp = f.readlines()

In [2]:
def passport_gen():
    passport = ""
    for line in inp:
        if line == "\n":
            yield passport
            passport = ""
        else:
            passport += line
    yield passport

def get_passport_dict(passport):
    p = " ".join(passport.split("\n")).strip() # Replace \n separator with spaces
    passport_dict = {}
    for param in p.split(" "):
        key, value = param.split(":")
        passport_dict[key] = value
    return passport_dict
    
    
def is_valid(passport, required_fields):
    passport_dict = get_passport_dict(passport)    
    return all({field: (field in passport_dict.keys()) for field in required_fields}.values())
    

# Part 1

In [3]:
passports = passport_gen()

required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
print(sum([int(is_valid(pp, required_fields)) for pp in passports]))

219


# Part 2

In [56]:
import re

class Validator:
    def __init__(self, rules):
        self.fields = {}
        for name, rule in rules.items():
            self.fields[name] = Field(name, rule)
            
    def check(self, passport_dict, verbose=False):
        if verbose:
            print("**Checking...**")
            result = all([self.fields[key].check(passport_dict.get(key, ""), verbose) for key, value in self.fields.items()])
            print(f"=> Result: {result}")
            return result
        else:
            return all([self.fields[key].check(passport_dict.get(key, "")) for key, value in self.fields.items()])
            
            
class Field:
    def __init__(self, name, rules):
        self.name = name
        self.regex = rules['regex']
        if 'vmin' in rules:
            self.vmin = rules['vmin']
        if 'vmax' in rules:
            self.vmax = rules['vmax']
        if 'func' in rules:
            self.func = rules['func']
            
    def check(self, value, verbose=False):
        if verbose:
            print(f"{self.name}: {value}")
        try:
            if value == "":
                raise Exception("The field is mandatory")
            
            if not (match := re.match(self.regex, value)):
                raise Exception("The regex doesn't match")

            try:
                if self.vmin > int(value):
                    raise Exception("The value is less than minimal")
            except AttributeError:
                pass

            try:
                if self.vmax < int(value):
                    raise Exception("The value is more than maximal")
            except AttributeError:
                pass

            try:
                if not self.func(match):
                    raise Exception("The value doesn't validate the user-defined function")
            except AttributeError:
                pass

            return True
        except Exception as e:
            if verbose:
                print(e)
            return False
        
def height_validation(match):
    if match[2] == "in":
        if int(match[1]) < 59 or int(match[1]) > 76:
            return False
    elif match[2] == "cm":
        if int(match[1]) < 150 or int(match[1]) > 193:
            return False
    return True

validation_rules = {
    'byr': {'regex': '^\d{4}$', 'vmin': 1920 , 'vmax': 2002}, 
    'iyr': {'regex': '^\d{4}$', 'vmin': 2010 , 'vmax': 2020},
    'eyr': {'regex': '^\d{4}$', 'vmin': 2020 , 'vmax': 2030},  
    'hgt': {'regex': '^(\d+)(cm|in)$', 'func': height_validation}, 
    'hcl': {'regex': '^\#[a-f0-9]{6}$'},  
    'ecl': {'regex': '^(amb|blu|brn|gry|grn|hzl|oth)$'}, 
    'pid': {'regex': '^\d{9}$'},
}


validator = Validator(validation_rules)


In [57]:
passports = passport_gen()

sum([validator.check(get_passport_dict(pp)) for pp in passports])

127